<img src="images/mcg.jpg", style="width: 100px">

# Bayesian Networks with PGMPY

We shall use the restaurant traffic model to determine how busy a restaurant is given that a new dish has been introduced.

<img src="images/restaurant_traffic.png", style="width: 600px;">

pgmpy is an open source library for working with Probablistic Graphical Models. Install pgmpy with the command:

```python
pip install pgmpy
```

Verify pgmpy is installed:

In [1]:
!pip install pgmpy

## Create a Bayesian Network Object

### Specifying the Structure of the Model

To create a Bayesian Network Model, you need to specify Nodes (Random Variables) in the heirarchical order of influence. We know the dependency chain as:

* Festival, Game -> Traffic
* Sunny, Rain -> Pedestrian
* New Dish(Dose), Traffic, Pedestrian -> Restaurant (Vidyarthi Bhavan).

In [28]:
from pgmpy.models import BayesianModel

restaurant_model = BayesianModel([('F', 'T'),('G', 'T'), ('S', 'P'),
                                  ('R', 'P'), ('ND', 'B'), ('T', 'B'),
                                  ('P', 'B')])

We have now created nodes and directed edges of the Bayesian network.

### Specifying Conditional probabilites of each Nodes (Random Variables) 

The probability that there is an accident on the road leading to the restaurant is 0.05. Hence, we shall assign the CPDs accordingly. Simiarly, we shall assign CPDs for all variables:

Let us look at variables that can affect traffic patterns. These could be stochastic (time variant) as well, however for the current example, let us assume the september season with Krishna janmashtami and Gauri/Ganesha and assign a higher probability for a festival.

* Festival (boolean states) 

|      CPD               |    No      |    Yes   |
|------------------------|------------|----------|     
| Festival               |    0.7     |    0.3   | 

* CPD of Game

|      CPD               |    No      |    Yes   |
|------------------------|------------|----------|     
| Game.                  |   0.95     |    0.05  | 

* CPD of Sunny status:


|      CPD               |    No      |    Yes   |
|------------------------|------------|----------|     
|      Sunny             |   0.7      |    0.3   | 

Similarly you can specify all CPDs.

In [36]:
from pgmpy.factors.discrete import TabularCPD
from pgmpy.models import BayesianModel

festival_cpd = TabularCPD(variable='F',
                       variable_card=2,
                       values=[[.7, .3]])

game_cpd = TabularCPD(variable='G',
                       variable_card=2,
                       values=[[.95, .05]])

sunny_cpd = TabularCPD(variable='S',
                       variable_card=2,
                       values=[[.7, .3]])

rain_cpd = TabularCPD(variable='R',
                       variable_card=2,
                       values=[[.8, .2]])

new_dish_cpd = TabularCPD(variable='N',
                       variable_card=2,
                       values=[[.7, .3]])


traffic_cpd = TabularCPD(variable='T',
                         variable_card=2,
                         values=[[.8, .05, .1, .5 ],
                                [.2,.95, .9, .5]],
                         evidence=['F', 'G'],
                         evidence_card=[2, 2])

pedestrian_cpd = TabularCPD(variable='P',
                         variable_card=2,
                         values=[[.4, .9, .1, .8 ],
                                [.6,.1, .9, .2]],
                         evidence=['S', 'R'],
                         evidence_card=[2, 2])                
                                 
Restaurant_booking_cpd = TabularCPD(variable='RB',
                         variable_card=2,
                         values=[[.95, .6, .9, .5, .4, .05, .7, .3 ],
                                [.05, .4, .1, .5, .6, .95, .3, .7]],
                         evidence=['N', 'T','P'],
                         evidence_card=[2, 2, 2])           

## Create a Bayesian Model

Now that we have CPDs, create a bayesian model specifying all the dependencies as evidence.

In [37]:
restaurant_model = BayesianModel([('F', 'T'),('G', 'T'),
                                  ('S', 'P'),('R', 'P'),
                                  ('T','RB'),('N','RB'),
                                  ('P','RB')])

## Add CPDs

Add all cpds to the restaurant model, validate the model and iterate over the CPDs to verify.

In [38]:
restaurant_model.add_cpds(festival_cpd, game_cpd,
                          sunny_cpd, rain_cpd,
                          new_dish_cpd, traffic_cpd, 
                          pedestrian_cpd, Restaurant_booking_cpd)

for cpd in restaurant_model.get_cpds():
    print("CPD of {variable}:".format(variable=cpd.variable))
    print(cpd) 

CPD of F:
╒═════╤═════╕
│ F_0 │ 0.7 │
├─────┼─────┤
│ F_1 │ 0.3 │
╘═════╧═════╛
CPD of G:
╒═════╤══════╕
│ G_0 │ 0.95 │
├─────┼──────┤
│ G_1 │ 0.05 │
╘═════╧══════╛
CPD of S:
╒═════╤═════╕
│ S_0 │ 0.7 │
├─────┼─────┤
│ S_1 │ 0.3 │
╘═════╧═════╛
CPD of R:
╒═════╤═════╕
│ R_0 │ 0.8 │
├─────┼─────┤
│ R_1 │ 0.2 │
╘═════╧═════╛
CPD of N:
╒═════╤═════╕
│ N_0 │ 0.7 │
├─────┼─────┤
│ N_1 │ 0.3 │
╘═════╧═════╛
CPD of T:
╒═════╤═════╤══════╤═════╤═════╕
│ F   │ F_0 │ F_0  │ F_1 │ F_1 │
├─────┼─────┼──────┼─────┼─────┤
│ G   │ G_0 │ G_1  │ G_0 │ G_1 │
├─────┼─────┼──────┼─────┼─────┤
│ T_0 │ 0.8 │ 0.05 │ 0.1 │ 0.5 │
├─────┼─────┼──────┼─────┼─────┤
│ T_1 │ 0.2 │ 0.95 │ 0.9 │ 0.5 │
╘═════╧═════╧══════╧═════╧═════╛
CPD of P:
╒═════╤═════╤═════╤═════╤═════╕
│ S   │ S_0 │ S_0 │ S_1 │ S_1 │
├─────┼─────┼─────┼─────┼─────┤
│ R   │ R_0 │ R_1 │ R_0 │ R_1 │
├─────┼─────┼─────┼─────┼─────┤
│ P_0 │ 0.4 │ 0.9 │ 0.1 │ 0.8 │
├─────┼─────┼─────┼─────┼─────┤
│ P_1 │ 0.6 │ 0.1 │ 0.9 │ 0.2 │
╘═════╧═════╧═════╧═══

## Inference from the Model

To obtain probabilistic inference from the model given various scenarios, use the inference class.

In [39]:
from pgmpy.inference.base import Inference
from pgmpy.factors import factor_product

import itertools

class SimpleInference(Inference):
    def query(self, var, evidence):
        # self.factors is a dict of the form of {node: [factors_involving_node]}
        factors_list = set(itertools.chain(*self.factors.values()))
        product = factor_product(*factors_list)
        reduced_prod = product.reduce(evidence, inplace=False)
        reduced_prod.normalize()
        var_to_marg = set(self.model.nodes()) - set(var) - set([state[0] for state in evidence])
        marg_prod = reduced_prod.marginalize(var_to_marg, inplace=False)
        return marg_prod

## Case Study

### How Busy is the Restaurant?

* Given New dish is introduced during a festival and it is a sunny day.

In [43]:
infer = SimpleInference(restaurant_model)

l1 = infer.query(var=['RB'], evidence=[('N', 0), ('F', 0), ('S', 0)])
print(l1)

╒══════╤═══════════╕
│ RB   │   phi(RB) │
╞══════╪═══════════╡
│ RB_0 │    0.7572 │
├──────┼───────────┤
│ RB_1 │    0.2428 │
╘══════╧═══════════╛


We see that the restaurant is quite busy!

* Given that the restaurant is busy and no dish was introduced, can we predict how likely is it to be a sunny day?

In [44]:
l2 = infer.query(var=['S'], evidence=[('RB', 0), ('N', 1)])
print(l2)

╒═════╤══════════╕
│ S   │   phi(S) │
╞═════╪══════════╡
│ S_0 │   0.7645 │
├─────┼──────────┤
│ S_1 │   0.2355 │
╘═════╧══════════╛


We see that the day is very likely to be a sunny day!